In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

from apscheduler.schedulers.background import BackgroundScheduler

from contextlib import contextmanager

from rich.console import Console
from rich.table import Table
from rich.live import Live

from getpass import getpass

from datetime import datetime, timezone
import re
import random
import requests
import subprocess
import time

console = Console()

RESOURCE_FIELDS = 'resource_fields'
ADVENTURES = 'adventures'
RAIDS = 'raids'
logs = {}

In [3]:
''' web driver init. '''

def init_webdriver(proxy_port=None):
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")

    # Disable image loading
    #prefs = {"profile.managed_default_content_settings.images": 2}
    #options.add_experimental_option("prefs", prefs)

    # proxy options
    if proxy_port:
        proxy_address = f'localhost:{proxy_port}'
        options.add_argument(f'--proxy-server={proxy_address}')

    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [4]:
''' scheduler init. '''

def calc_new_interval_between(x, y):
    return random.uniform(x, y)

def cleanup_scheduler(scheduler):
    if scheduler.running:
        print("Shutting down scheduler...")
        scheduler.shutdown(wait=True)
    else:
        print("Scheduler already stopped.")

# Context manager for BackgroundScheduler
@contextmanager
def managed_scheduler(*args, **kwargs):
    scheduler = BackgroundScheduler(*args, **kwargs)
    scheduler.start()
    try:
        yield scheduler
    finally:
        cleanup_scheduler(scheduler)

In [5]:
''' Open Travian International 2 server and login if not already '''

def attempt_login(site, username, password, driver):
    driver.get(site)

    try:
        input_username = driver.find_element(By.XPATH, '//*[@id="loginForm"]/tbody/tr[1]/td[2]/input')
        input_pwd = driver.find_element(By.XPATH, '//*[@id="loginForm"]/tbody/tr[2]/td[2]/input')
        button_login = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"][value="Login"].textButtonV1.green')

        input_username.send_keys(username)
        input_pwd.send_keys(password)
        button_login.click()
    except NoSuchElementException:
        print("User already signed in!")

In [6]:
''' Send hero on adventure '''

def attempt_to_start_adventure(driver, scheduler):
    button_hero_status = driver.find_element(By.XPATH, '//*[@id="topBarHero"]/div/a/i')

    # dynamic ID, using href to reference
    button_adventures = driver.find_element(By.XPATH, '//a[contains(@href, "/hero/adventures")]')
    try:
        num_adventures = button_adventures.find_element(By.XPATH, "./div")
    except NoSuchElementException:
        num_adventures = None

    if num_adventures and 'heroHome' in button_hero_status.get_attribute('class') and int(num_adventures.text) > 0:
        button_adventures.click()

        button_start_first_adventure = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="heroAdventure"]/table/tbody/tr[1]/td[5]/button')))
        button_start_first_adventure.click()

        button_continue = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="heroAdventure"]/div/button')))
        button_continue.click()

        logs[f'{driver.session_id}_{ADVENTURES}'] = "Successfully sent out hero on adventure!"
    elif num_adventures is None:
        logs[f'{driver.session_id}_{ADVENTURES}'] = "No adventures found."
    elif 'heroRunning' in button_hero_status.get_attribute('class'):
        logs[f'{driver.session_id}_{ADVENTURES}'] = "Hero is already on an adventure!"
    else:
        logs[f'{driver.session_id}_{ADVENTURES}'] = "Error attempting to start adventure."
    
    scheduler.add_job(attempt_to_start_adventure, 'interval', seconds=calc_new_interval_between(307, 902),
                      id=f'{driver.session_id}_{ADVENTURES}', args=[driver, scheduler],
                      replace_existing=True)

In [7]:
def disable_contextual_help(driver):
    button_options = driver.find_element(By.XPATH, '//a[contains(@href, "/options")]')
    driver.execute_script("arguments[0].click();", button_options) # use javascript to get around helper popup

    checkbox_contextual_help = driver.find_element(By.XPATH, '//*[@id="hideContextualHelp"]')
    checkbox_contextual_help.click()
    
    save_button = driver.find_element(By.CSS_SELECTOR, '.textButtonV1.green')
    driver.execute_script("arguments[0].scrollIntoView(true);", save_button)
    save_button.click()


def dismiss_report_helper_popup(driver):
    button_reports = driver.find_element(By.XPATH, '//*[@id="navigation"]/a[5]')
    driver.execute_script("arguments[0].click();", button_reports) # use javascript to get around helper popup


def dismiss_ok_popup(driver):
    try:
        button_ok = driver.find_element(By.XPATH, '//*[@id="contextualHelp"]/div/div[2]/nav/button')
        driver.execute_script("arguments[0].click();", button_ok) # use javascript to get around helper popup
    except NoSuchElementException:
        pass


In [8]:
def enter_buildings_page(driver):
    if 'dorf2.php' not in driver.current_url:
        button_buildings_page = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="navigation"]/a[2]')))
        button_buildings_page.click()

        dismiss_ok_popup(driver)


def can_build(building_slot):
    if 'good' not in building_slot.get_attribute('class'):
        print("Unable to upgrade, please ensure you have enough resources and building queue is not full.")
        return False
    return True   


def press_construct_building_button_for(driver, building):
    try:
        header = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, f"//h2[text()='{building}']")))
    except TimeoutException:
        print("Unable to find building! Has it already been constructed or are you on the wrong building tab?")
        return False

    building_wrapper = header.find_element(By.XPATH, '..')
    build_button = building_wrapper.find_element(By.CSS_SELECTOR, '.textButtonV1.green.new')
    build_button.click()
    return True

def construct_building_in_slot(driver, building, building_slot):
    enter_buildings_page(driver)

    building_slot = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, f"//a[contains(@href, '/build.php?id={building_slot}')]")))
    if not can_build(building_slot):
        return

    driver.execute_script("arguments[0].click();", building_slot) # use javascript to get around helper popup

    for i in range(1, 4): # check all building tabs for building
        try:
            infrastructure_tab = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, f"//a[contains(@href, '/build.php?id={building_slot}&category={i}')]")))
            infrastructure_tab.click()
        except TimeoutException: # if tab not found we have another helper popup, dismiss it then retry
            popup_next_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//*[@id='contextualHelp']/div/div[2]/nav/button")))
            driver.execute_script("arguments[0].click();", popup_next_button)
            driver.execute_script("arguments[0].click();", popup_next_button)

        began_constructing = press_construct_building_button_for(driver, building)
        if began_constructing:
            break

    print(f"Successfully began constructing {building} in slot {building_slot}")


def wall_built(driver):
    enter_buildings_page(driver)

    building_slot = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='villageContent']/div[22]"))) # wall always in slot 40

    if building_slot.get_attribute('data-name'):
        return True
    return False


def construct_wall(driver):
    enter_buildings_page(driver)

    building_slot = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//a[contains(@href, '/build.php?id=40')]"))) # wall always in slot 40
    if not can_build(building_slot):
        return

    driver.execute_script("arguments[0].click();", building_slot) # use javascript to get around helper popup

    # wall is the only building allowed in slot 40, build button for it will be the only one that exists on the page
    try:
        build_button = driver.find_element(By.CSS_SELECTOR, '.textButtonV1.green.new')
        build_button.click()
    except NoSuchElementException:
        print("Wall has already been constructed!")

In [9]:
def enter_resource_fields_page(driver):
    if 'dorf1.php' not in driver.current_url:
        button_resource_fields_page = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="navigation"]/a[1]')))
        button_resource_fields_page.click()

        dismiss_ok_popup(driver)


def upgrade_mission_clay_field(driver):
    enter_resource_fields_page(driver)

    building_slot = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//a[contains(@href, '/build.php?id=5')]")))
    if not can_build(building_slot):
        return

    driver.execute_script("arguments[0].click();", building_slot) # use javascript to get around helper popup

    button_upgrade = driver.find_element(By.CSS_SELECTOR, '.textButtonV1.green.build')
    if '2' in button_upgrade.text:
        button_upgrade.click()
    else:
        print("Clay field already at lvl 2.")


WOOD = 1
CLAY = 2
IRON = 3
WHEAT = 4
def find_lowest_level_field_of_type(driver, gid):
    enter_resource_fields_page(driver)

    resource_field_container = driver.find_element(By.XPATH, '//*[@id="resourceFieldContainer"]')
    resource_field_links = resource_field_container.find_elements(By.TAG_NAME, 'a')

    target_fields = []
    for link in resource_field_links:
        if 'good' in link.get_attribute('class') and f'gid{gid}' in link.get_attribute('class'):
            target_fields.append(link)

    lowest_level_field = None
    lowest_level = 999

    for field in target_fields:
        if 'underConstruction' in field.get_attribute('class'):
            continue

        curr_level = int(field.find_element(By.XPATH, './div').text.strip() or 0)
        if curr_level < lowest_level:
            lowest_level_field = field
            lowest_level = curr_level

    return lowest_level_field


def attempt_to_upgrade_lowest_level_field(driver, scheduler):
    enter_resource_fields_page(driver)

    lowest_level_field = None
    lowest_level = 999
    gid_types = [WOOD, CLAY, IRON, WHEAT]  # List of gid types you want to check

    for gid in gid_types:
        field = find_lowest_level_field_of_type(driver, gid)
        if field:
            curr_level = int(field.find_element(By.XPATH, './div').text.strip() or 0)
            if curr_level < lowest_level:
                lowest_level_field = field
                lowest_level = curr_level

    if lowest_level_field:
        lowest_level_field.click()
        button_upgrade = driver.find_element(By.CSS_SELECTOR, '.textButtonV1.green.build')
        button_upgrade.click()

        logs[f'{driver.session_id}_{RESOURCE_FIELDS}'] = "Began upgrading resource field."
    else:
        logs[f'{driver.session_id}_{RESOURCE_FIELDS}'] = "No upgradable fields were found!"

    scheduler.add_job(attempt_to_upgrade_lowest_level_field, 'interval', seconds=calc_new_interval_between(907, 2403), id=f'{driver.session_id}_{RESOURCE_FIELDS}', args=[driver, scheduler], replace_existing=True)

In [10]:
def run_missions_and_disable_contextual_helpers(driver):
    disable_contextual_help(driver)
    upgrade_mission_clay_field(driver)
    construct_building_in_slot(driver, 'Cranny', 30)
    construct_wall(driver)
    dismiss_report_helper_popup(driver)
    dismiss_ok_popup(driver)

In [11]:
def check_total_troop_counts():
    pass


def activate_farm_list_raids_for(list_element, driver):
    name = WebDriverWait(list_element, 10).until(EC.presence_of_element_located(
        (By.XPATH, './div/div[1]/div[2]/div[1]'))).text

    table = list_element.find_element(By.XPATH, './div/div[2]/table')
    table_body = table.find_element(By.XPATH, './tbody')
    table_footer = table.find_element(By.XPATH, './tfoot')
    table_rows = table_body.find_elements(By.TAG_NAME, 'tr')

    raids_possible = False
    for row in table_rows:
        try:
            checkbox = row.find_element(By.XPATH, './td[1]/label/input')
            driver.execute_script("arguments[0].scrollIntoView(true);", checkbox)
        except NoSuchElementException:
            continue

        last_raid_state_container = row.find_element(By.XPATH, './td[7]')
        try:
            last_raid_state = last_raid_state_container.find_element(By.TAG_NAME, 'i')
        except NoSuchElementException:
            last_raid_state = None
        
        curr_state_container = row.find_element(By.XPATH, './td[2]')
        try:
            curr_state = curr_state_container.find_element(By.TAG_NAME, 'i')
        except NoSuchElementException:
            curr_state = None

        num_troops_container = row.find_element(By.XPATH, './td[6]')
        num_troops = int(num_troops_container.find_element(By.XPATH, './div[1]/span[1]/span[1]').text)

        try:
            filthy_nominator   = table_footer.find_element(By.XPATH, './tr[1]/td[2]/div/div/span/span/span[1]').text
            filthy_denominator = table_footer.find_element(By.XPATH, './tr[1]/td[2]/div/div/span/span/span[2]').text
            nominator   = int(re.sub(r'[^\d]', '', filthy_nominator))
            denominator = int(re.sub(r'[^\d]', '', filthy_denominator))
        except NoSuchElementException:
            nominator = denominator = None

        if (not curr_state or 'attack_small' not in curr_state.get_attribute('class')) and \
        (not last_raid_state or 'attack_won_withoutLosses_small' in last_raid_state.get_attribute('class')) and \
        ((not nominator and not denominator) or nominator + num_troops <= denominator):
            raids_possible = True
            checkbox.click()

    # start raids
    if raids_possible:
        button_start_raids = list_element.find_element(By.XPATH, './div/div[1]/button')
        driver.execute_script("arguments[0].scrollIntoView(true);", button_start_raids)
        button_start_raids.click()

    logs[f'{driver.session_id}_{RAIDS}'] = f'Finished raid logic for {name}'


def send_troops_to_farm(driver, scheduler):
    if 'build.php?id=39&gid=16&tt=99' not in driver.current_url:
        enter_buildings_page(driver)

        button_rally_point = driver.find_element(By.XPATH, '//*[@id="villageContent"]/div[21]/a')
        button_rally_point.click()

        try:
            button_farm_list = driver.find_element(By.XPATH, '//a[contains(@href, "/build.php?id=39&gid=16&tt=99")]')
            button_farm_list.click()
        except NoSuchElementException:
            logs[f'{driver.session_id}_{RAIDS}'] = "Please activate Travian Gold Club to gain access to farm lists."
            return


    try:
        farm_lists_container = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.villageWrapper')))
    except TimeoutException:
        logs[f'{driver.session_id}_{RAIDS}'] = "Please create a farm list to begin raiding!"
        return

    farm_lists = farm_lists_container.find_elements(By.CSS_SELECTOR, '.dropContainer')
    for farm_list in farm_lists:
        activate_farm_list_raids_for(farm_list, driver)

    logs[f'{driver.session_id}_{RAIDS}'] = "Finished raid attempt."

    scheduler.add_job(attempt_to_start_adventure, 'interval', seconds=calc_new_interval_between(183, 302),
                      id=f'{driver.session_id}_{RAIDS}', args=[driver, scheduler],
                      replace_existing=True)

In [12]:
# TODO: Update table generation to handle multiple drivers
def generate_job_scheduler_table_from(scheduler):
    table = Table(title="Scheduled Jobs")

    table.add_column("Job ID", style="cyan", no_wrap=True)
    table.add_column("Next Run At", style="magenta")
    table.add_column("Countdown", style="green")
    table.add_column("Log", style="blue")

    now = datetime.now(timezone.utc)
    for job in scheduler.get_jobs():
        next_run = job.next_run_time
        countdown = (next_run - now).total_seconds()
        table.add_row(job.id, str(next_run), str(int(countdown)), logs.get(job.id, ''))
    
    return table

In [15]:
''' init. and login to server (single account) '''
input_site = 'https://ts2.x1.international.travian.com/dorf1.php'
#input_site = input("Site:") # Commented out while testing

input_username = input("Username:")
input_password = getpass("Password:")

driver = init_webdriver() # !!! NO PROXY ADDED, ENSURE PROXY PORT USED TO SETUP ACCOUNT IS ADDED HERE TO AVOID BANS
attempt_login(input_site, input_username, input_password, driver)

# !!! STILL IN TESTING, will fail without restarting once building queue is full
#if not wall_built(driver):
#    run_missions_and_disable_contextual_helpers(driver)

''' begin scheduler tasks '''
with managed_scheduler() as scheduler:
    scheduler.add_job(attempt_to_start_adventure, 'interval', seconds=10,
                        id=f'{driver.session_id}_{ADVENTURES}', args=[driver, scheduler])
    scheduler.add_job(send_troops_to_farm, 'interval', seconds=20,
                        id=f'{driver.session_id}_{RAIDS}', args=[driver, scheduler])
    scheduler.add_job(attempt_to_upgrade_lowest_level_field, 'interval', seconds=60,
                        id=f'{driver.session_id}_{RESOURCE_FIELDS}', args=[driver, scheduler])

    with Live(generate_job_scheduler_table_from(scheduler), refresh_per_second=1, console=console) as live:
        try:
            while True:
                live.update(generate_job_scheduler_table_from(scheduler))
                time.sleep(1)
        except (KeyboardInterrupt, SystemExit):
            pass

Output()

In [14]:
''' init. and login to server (multiple accounts) '''
input_site = 'https://ts2.x1.international.travian.com/dorf1.php'
#input_site = input("Site:") # Commented out while testing

num_accounts = 5
drivers = []
user_info = {}
for i in range(num_accounts): # get accounts info
    proxy_port = 24000 + i

    user_info[proxy_port] = {}
    user_info[proxy_port]['username'] = input(f"Username for proxy port {proxy_port}:")
    user_info[proxy_port]['password'] = getpass(f"Password for proxy port {proxy_port}:")


for i in range(num_accounts): # initialize web drivers
    proxy_port = 24000 + i

    driver = init_webdriver(proxy_port)
    attempt_login(input_site, user_info[proxy_port]['username'], user_info[proxy_port]['password'], driver)

    # !!! STILL IN TESTING, will fail without restarting once building queue is full
    #if not wall_built(driver):
    #    run_missions_and_disable_contextual_helpers(driver)
    
    drivers.append(driver)


''' begin scheduler tasks '''
with managed_scheduler() as scheduler:
    for driver in drivers:
        scheduler.add_job(attempt_to_start_adventure, 'interval', seconds=10,
                          id=f'{driver.session_id}_{ADVENTURES}', args=[driver, scheduler])
        scheduler.add_job(send_troops_to_farm, 'interval', seconds=20,
                          id=f'{driver.session_id}_{RAIDS}', args=[driver, scheduler])
        scheduler.add_job(attempt_to_upgrade_lowest_level_field, 'interval', seconds=30,
                          id=f'{driver.session_id}_{RESOURCE_FIELDS}', args=[driver, scheduler])
    

    with Live(generate_job_scheduler_table_from(scheduler), refresh_per_second=1, console=console) as live:
        try:
            while True:
                live.update(generate_job_scheduler_table_from(scheduler))
                time.sleep(1)
        except (KeyboardInterrupt, SystemExit):
            pass

Output()

Shutting down scheduler...


In [ ]:
%%bash
curl -X POST "http://127.0.0.1:22999/api/shutdown" # used to shutdown proxy-manager

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    15  100    15    0     0   1350      0 --:--:-- --:--:-- --:--:--  2142


{"result":"ok"}